# Treinamento Sem card counting

Dataset: https://www.kaggle.com/datasets/dennisho/blackjack-hands/

Modelo: Regressão logística (sem o uso de card counting)

## 1) Tratamento dos dados

In [14]:
import pandas as pd

# Número de linhas a serem consideradas
rows = 25000

df = pd.read_csv('blackjack_simulator.csv', sep=',', nrows=rows)

# Remove as colunas não utilizadas
df = df.drop(columns=['shoe_id', 'run_count', 'true_count', 'cards_remaining', 'dealer_up', 'initial_hand', 'dealer_final_value', 'player_final_value'])

# Filtra os dados por ganho
df = df[df['win'] >= 1]

# Remove as jogadas que não são utilizadas na simulação
df = df[~df['actions_taken'].astype(str).str.contains('N')]

# Transforma as strings em listas
df['dealer_final'] = df['dealer_final'].apply(lambda x: eval(x)[0])  # Só a primeira carta do dealer será utilizada para emular as informações que um jogador tem
df['player_final'] = df['player_final'].apply(lambda x: eval(x))
df['actions_taken'] = df['actions_taken'].apply(lambda x: eval(x))

rows = len(df)

df

,dealer_final,player_final,actions_taken,win
0,10,"[[10, 11]]",[[S]],1.5
1,10,"[[5, 5, 11]]","[[H, S]]",1.0
2,6,"[[3, 10]]",[[S]],1.0
6,6,"[[3, 2, 10], [3, 4, 5]]","[[P, H, S], [H, S]]",2.0
8,5,"[[8, 10]]",[[S]],1.0
...,...,...,...,...
24988,3,"[[6, 9]]",[[S]],1.0
24992,8,"[[8, 6, 4]]","[[H, S]]",1.0
24993,10,"[[10, 4, 4]]","[[H, S]]",1.0
24998,10,"[[10, 10]]",[[S]],1.0


In [15]:
# Divide as ações dos players em vários estados de jogo
for i in range(0, len(df.index)):
    linha = df.loc[i]

    for j in range(0, len(linha['actions_taken'])):
        mao = linha['player_final'][j]
        acoes = linha['actions_taken'][j]

        # print({'dealer_final': linha['dealer_final'], 'player_final': mao[:4], 'actions_taken': acoes[0]})

        for k in range(0, len(acoes)):
            acao = acoes[k]

            df = pd.concat([df, pd.DataFrame([{'dealer_final': linha['dealer_final'], 'player_final': sum(mao[:2+k]), 'actions_taken': acoes[k], 'win': linha['win']}])], ignore_index=True)

df = df.drop(range(0, rows))
df


,dealer_final,player_final,actions_taken,win
10093,10,21,S,1.5
10094,10,10,H,1.0
10095,10,21,S,1.0
10096,6,13,S,1.0
10097,6,5,P,2.0
...,...,...,...,...
24305,8,18,S,1.0
24306,10,14,H,1.0
24307,10,18,S,1.0
24308,10,20,S,1.0


## 2) Treinamento

### Regressão Logística

In [16]:
# from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import numpy as np

X = df.drop(['actions_taken'], axis=1)
y = df['actions_taken']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# clf = MLPClassifier().fit(X_train, y_train)
# clf.score()
clf_reg_log = LogisticRegression(max_iter=2000, random_state=0).fit(X_train[['dealer_final', 'player_final']], y_train, sample_weight=X_train['win'].values)
clf_reg_log.score(X_test.drop('win', axis=1), y_test)

0.7834036568213784

#### Probabilidade de realizar cada jogada

In [17]:
# Estado do jogo
X_test = X_test.drop('win', axis=1)
X_test[:1]

,dealer_final,player_final
19647,7,13


In [18]:
# Probabilidade de realizar cada jogada
classes = clf_reg_log.classes_
probabilidade = clf_reg_log.predict_proba(X_test[:1])[0]

for i in range(0, len(classes)):
    print(f'JOGADA: {classes[i]} {probabilidade[i]*100.0:.02f} %')

JOGADA: D 28.33 %
JOGADA: H 39.83 %
JOGADA: P 5.71 %
JOGADA: S 26.13 %


In [19]:
# Jogada a ser realizada
clf_reg_log.predict(X_test[:1])[0]

'H'

### Árvore de Decisão

In [20]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
import numpy as np

X = df.drop(['actions_taken'], axis=1)
y = df['actions_taken']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

clf_arvore = DecisionTreeClassifier().fit(X_train[['dealer_final', 'player_final']], y_train, sample_weight=X_train['win'].values)
# clf_reg_log = LogisticRegression(max_iter=2000, random_state=0).fit(X_train[['dealer_final', 'player_final']], y_train, sample_weight=X_train['win'].values)
# clf_reg_log.score(X_test.drop('win', axis=1), y_test)
clf_arvore.score(X_test.drop('win', axis=1), y_test)

0.9194796061884669

#### Probabilidade de realizar cada jogada

In [21]:
# Estado do jogo
X_test = X_test.drop('win', axis=1)
X_test[:1]

,dealer_final,player_final
18782,6,13


In [22]:
# Probabilidade de realizar cada jogada
classes = clf_arvore.classes_
probabilidade = clf_arvore.predict_proba(X_test[:1])[0]

for i in range(0, len(classes)):
    print(f'JOGADA: {classes[i]} {probabilidade[i]*100.0:.02f} %')

JOGADA: D 19.01 %
JOGADA: H 0.00 %
JOGADA: P 9.09 %
JOGADA: S 71.90 %


In [23]:
# Jogada a ser realizada
clf_arvore.predict(X_test[:1])[0]

'S'

## 3) Exportação

In [24]:
import pickle

pickle.dump(clf_reg_log, open('out/reglog.pkl', 'wb'))
pickle.dump(clf_arvore, open('out/arvore.pkl', 'wb'))